In [2]:
import sys
sys.path.append('../../..')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import feature_column as fc
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn import preprocessing
from tensorflow.keras import optimizers
import tensorflow.keras.backend as K

from utils.gpu.cuda_cluster import *
from utils.gpu.preprocessing import read_data, factorize_small_cardinality_with_index, df_to_tfdataset, split_join, get_media_index
from utils.evaluate import calculate_ctr, compute_rce, average_precision_score
import core.config as conf

In [3]:
client

Client Scheduler: tcp://127.0.0.1:32843 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 33.47 GB


## Load data

In [4]:
# data_path = conf.raw_data_path + '*' # for all dataset
data_path = conf.data_root + 'dask_input'
df = dask_cudf.read_parquet(f'{data_path}/train-final-te-like-1.parquet', index=False)

## Preprocessing

In [5]:
df.head()

,timestamp,creator_follower_count,creator_following_count,creator_is_verified,creator_account_creation,engager_follower_count,engager_following_count,engager_is_verified,engager_account_creation,engager_follows_creator,...,TE_creator_count_combined_tweet_type_language_like,TE_creator_user_fer_count_delta_time_media_language_like,TE_creator_user_fing_count_delta_time_media_language_like,TE_creator_user_fering_count_delta_time_tweet_type_language_like,TE_creator_user_fing_count_mode_media_language_like,TE_creator_user_fer_count_mode_media_language_like,TE_creator_user_fering_count_mode_tweet_type_language_like,TE_domains_media_tweet_type_language_like,TE_links_media_tweet_type_language_like,TE_hashtags_media_tweet_type_language_like
0,1612587832,3885,3051,0,1568428850,634,531,0,1590421816,1,...,0.472310,0.353791,0.353791,0.472310,0.354039,0.354039,0.471939,0.435391,0.435391,0.423956
1,1613743226,226443,0,0,1597609757,633,151,0,1541162905,0,...,0.474047,0.355000,0.355000,0.474047,0.355901,0.355901,0.473944,0.432625,0.432625,0.434331
2,1613369374,1353309,537,1,1299819150,2304,844,0,1278714864,0,...,0.527204,0.488510,0.488510,0.527204,0.487882,0.487882,0.526625,0.597564,0.597564,0.601652
3,1614038010,226308,9,0,1468552079,85,808,0,1387326144,0,...,0.524442,0.475492,0.475492,0.523578,0.475356,0.475356,0.524442,0.597564,0.597564,0.601652
4,1612525320,131219,1023,0,1517575079,141,340,0,1560679572,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [6]:
df.isnull().sum().compute()

timestamp                                                          0
creator_follower_count                                             0
creator_following_count                                            0
creator_is_verified                                                0
creator_account_creation                                           0
                                                               ...  
TE_creator_user_fer_count_mode_media_language_like            146676
TE_creator_user_fering_count_mode_tweet_type_language_like    146222
TE_domains_media_tweet_type_language_like                      79182
TE_links_media_tweet_type_language_like                       308353
TE_hashtags_media_tweet_type_language_like                    340310
Length: 72, dtype: uint64

In [8]:
df = df.fillna(0)

In [9]:
df = df.astype(np.int64)

## Sampling

In [1]:
df_positive = df[df['reply']==1]
df_negative = df[df['reply']==0]

NameError: name 'df' is not defined

## Loss Function

In [18]:
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

## Modeling

In [20]:
df_y = df[['like']]
df_x = df.drop(['like'], axis = 1)

In [21]:
X_train = df_x
y_train = df_y['like'].to_frame()

## Scaling

In [22]:
standard_scaler = preprocessing.StandardScaler()
standard_scaler.fit(X_train.compute().as_matrix())

StandardScaler()

In [23]:
ss = standard_scaler.transform(X_train.compute().as_matrix())

In [24]:
X_train = pd.DataFrame(ss, columns = df_x.columns)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train.compute().to_pandas(), test_size=0.2)

In [26]:
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.2)

In [27]:
model = Sequential([
    Dense(64, activation = 'relu', input_dim = X_test.shape[1]),
    Dense(32, activation = 'relu'),
    Dense(16, activation = 'relu'),
    Dense(1, activation = 'sigmoid')
])

In [33]:
model.compile(
    optimizer = 'adam',
    loss = f1_loss, # softmax : sparse_categorical_crossentropy, sigmoid : binary_crossentropy
    metrics=['accuracy', f1])

In [34]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 64)                4608      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_6 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 17        
Total params: 7,233
Trainable params: 7,233
Non-trainable params: 0
_________________________________________________________________


In [35]:
y_train = y_train.astype(np.float32)
y_val = y_val.astype(np.float32)

In [36]:
model.fit(
    x = X_train,
    y = y_train,
    validation_data=(X_val, y_val),
    epochs=5,
    batch_size=64
)

Epoch 1/5
38503/38503 [==============================] - 19s 489us/step - loss: 0.3899 - accuracy: 0.5847 - f1: 0.6103 - val_loss: 0.3783 - val_accuracy: 0.6166 - val_f1: 0.6220
Epoch 2/5
38503/38503 [==============================] - 19s 499us/step - loss: 0.3796 - accuracy: 0.6193 - f1: 0.6204 - val_loss: 0.3826 - val_accuracy: 0.6323 - val_f1: 0.6174
Epoch 3/5
38503/38503 [==============================] - 19s 500us/step - loss: 0.3790 - accuracy: 0.6161 - f1: 0.6210 - val_loss: 0.3780 - val_accuracy: 0.6190 - val_f1: 0.6219
Epoch 4/5
38503/38503 [==============================] - 19s 483us/step - loss: 0.3807 - accuracy: 0.6175 - f1: 0.6193 - val_loss: 0.3774 - val_accuracy: 0.6179 - val_f1: 0.6227
Epoch 5/5
38503/38503 [==============================] - 19s 493us/step - loss: 0.3801 - accuracy: 0.6128 - f1: 0.6200 - val_loss: 0.3799 - val_accuracy: 0.6148 - val_f1: 0.6200


In [ ]:
model.evaluate(X_test, y_test)

## Predict

In [24]:
predict_like = model.predict(X_test)

In [21]:
predict_like

array([[0.29271668],
       [0.54038197],
       [0.37923923],
       ...,
       [0.65456855],
       [0.4925698 ],
       [0.65168685]], dtype=float32)

In [22]:
rce_like = compute_rce(predict_like, y_test['like'])
rce_like

12.243130258995183

In [23]:
average_precision_score(y_test['like'], predict_like)

0.6245947659889121